# Lectura 31: Streaming API

In [ ]:
import polars as pl
from polars import col

import polars as pl
from polars import col

query = (
    pl.scan_parquet('./data/vuelos/vuelos.parquet')
    .filter(col('MONTH') > 6)
    .group_by('AIRLINE').agg(
        media_tiempo_aire=col('AIR_TIME').mean()
    )
)

vuelos_agregado = query.collect(streaming=True)

vuelos_agregado

In [ ]:
print(query.explain(streaming=True))

In [ ]:
query_2 = (
    pl.scan_parquet('./data/vuelos/vuelos.parquet')
    .with_columns(
        col('AIR_TIME').mean().over('AIR_LINE')
    )
)

print(query_2.explain(streaming=True))